In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('tweets.csv')
df.head()

,Unnamed: 0,Tweet
0,0.0,US Presidential Debate 2020: Joe Biden calls T...
1,1.0,"US Presidential Debate 2020: Trump, Biden spar..."
2,2.0,Screenshot from the first Presidential debate ...
3,3.0,US Presidential Debate 2020 in a nutshell. Emb...
4,4.0,US Presidential Debate 2020: Trump and Biden d...


In [3]:
df.tail()

,Unnamed: 0,Tweet
619,95.0,Just because the Dr was giving her all of you ...
620,96.0,"Jan 13, 2020 — #TRUMP2020 #TRUMP2020 #TRUMP202..."
621,97.0,"Aug 24, 2020 — . #Trump2020 #RNCConvention2020..."
622,98.0,"Aug 20, 2020 — Embedded video. 1:33. 1.1K view..."
623,99.0,"Dec 4, 2020 — Then you are lying to yourself! ..."


In [4]:
trump_refs = ['DonaldTrump','Donal Trump','Donald','Trump','TRUMP'] 
biden_refs = ['JoeBiden','Joe Biden','Joe','Biden','BIDEN']

In [5]:
def identify_subject(tweet,refs):
    flag =0
    for ref in refs:
        if tweet.find(ref)!=-1:
            flag=1
    return flag
df['Trump']=df['Tweet'].apply(lambda x:identify_subject(x,trump_refs))
df['Biden']=df['Tweet'].apply(lambda x:identify_subject(x,biden_refs))

In [6]:
import nltk
from nltk.corpus import stopwords
from textblob import Word,TextBlob

In [7]:
stop_words = stopwords.words('english')
custom_stopwords = {'RT','#presidentialDebate','#trumpence2020', '#bidenharris2020', '#debatenight', '#presidentialdebate2020',
                     '#2020election','#Vote', '#2020usaelection', '#Biden', '#BlueWave2020', '#donaldtrump',
'#Election2020', '#Elections_2020', '#MyPresident', '#November3rd','US','Presidential','Debate','#TRUMP2020','2020:'}

In [8]:
def preprocess_tweets(tweet,custom_stopwords):
    preprocessed_tweet=tweet
    preprocessed_tweet=tweet.replace('[^\w\s]','')
    preprocessed_tweet=" ".join(word for word in preprocessed_tweet.split() if word not in stop_words)
    preprocessed_tweet=" ".join(word for word in preprocessed_tweet.split() if word not in custom_stopwords)
    preprocessed_tweet=" ".join(Word(word).lemmatize() for word in preprocessed_tweet.split())
    return(preprocessed_tweet)

In [9]:
df['Processed Tweet'] = df['Tweet'].apply(lambda x: preprocess_tweets(x,custom_stopwords))

In [10]:
df['polarity'] = df['Processed Tweet'].apply(lambda x:TextBlob(x).sentiment[0])
df['subjectivity']=df['Processed Tweet'].apply(lambda x:TextBlob(x).sentiment[1])

In [11]:
display(df[df['Trump']==1][['Trump','polarity','subjectivity']].groupby('Trump').agg([np.mean, np.max, np.min, np.median]))
df[df['Biden']==1][['Biden','polarity','subjectivity']].groupby('Biden').agg([np.mean, np.max, np.min, np.median])

polarity                  subjectivity                 
           mean amax amin median         mean amax amin median
Trump                                                         
1      0.129082  1.0 -1.0    0.0     0.264863  1.0  0.0    0.0

polarity                  subjectivity                    
          mean amax amin median         mean amax amin    median
Biden                                                           
1      0.07615  0.8 -1.0    0.0     0.309375  1.0  0.0  0.066667